# Opening restaurant in Berlin

## 1. Introduction/Business Problem

> An exclusivist Canadian restauranteur that has a strong presence in North American HoReCa is looking to expand reach into European markets. After considering several Western European capitals for the opening of its first unit in the continent, the chain management has selected Berlin as the first stop. As part of a preliminary analysis of the HoReCa market, the management is looking at the main competition in the city, and where the competition is located. 
<br>
<br>
> In order to provide a complete overview of the competitive environment in Berlin, the analytic strategy is to select all restaurants in the city and cluster them based on popularity information, traffic statistics and pricing policies. A complete map of these clusters is to be presented to the management to gain a preliminary understanding of the market and potential customers.  


## 2. Data description

> A comprehensive listing of the restaurants in Berlin will be extracted from Foursquare based on Berlin latitude and longitude data. For each venue id extracted from the Foursquare database, a full overview of 
- unit popularity, 
- pricing information and 
- traffic statistics 

>... will be crawled from Foursquare database. 
 
> Namely, each venue ID will be associated with: a Category, CheckinsCount, usersCount, tipCount, rating, number of likes, number of dislikes, and price perceptions. Once the above variables are preprocessed (dummy coding, scaling where necessary/available), the restaurants in Berlin area will be clustered using Kmeans algorithm. The resulting clusters will be plotted on a map using Folium to depict areas with high competition and market potential. 

## Step 1: Importing the necessary libraries 

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize

Get the geographical coordinates of Berlin, Germany

In [2]:
address = 'Berlin, Germany'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Berlin, Germany are {}, {}.'.format(latitude, longitude))

C:\Users\Camelia Oancea\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinates of Berlin, Germany are 52.5170365, 13.3888599.


## Step 2: Getting the venues details from Foursquare

In [3]:
CLIENT_ID = '$%%#5' 
CLIENT_SECRET = '82399' 
VERSION = '20180605'
LIMIT = 100


In [4]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(CLIENT_ID,
CLIENT_SECRET,
VERSION,
latitude,
longitude,
LIMIT)

In [5]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bf91a93dd57976d449db527'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 821,
  'headerLocation': 'Unter den Linden',
  'headerFullLocation': 'Unter den Linden, Berlin',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 170,
  'suggestedBounds': {'ne': {'lat': 52.52316175832744,
    'lng': 13.400537482440706},
   'sw': {'lat': 52.5099222019844, 'lng': 13.376702263641915}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcda8ef964a520a74a21e3',
       'name': 'Dussmann das KulturKaufhaus',
       'location': {'address': 'Friedrichstr. 90',
        'lat': 52.51839819531867,
        'lng': 13.388788104057312,
        'labeledLa

In [6]:
venues = results['response']['groups'][0]['items']
berlin_venues = json_normalize(venues)

In [7]:
berlin_venues.head(5)

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,...,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4adcda8ef964a520a74a21e3-0,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",4adcda8ef964a520a74a21e3,Friedrichstr. 90,DE,Berlin,Deutschland,NaN,...,"[{'label': 'display', 'lat': 52.51839819531867...",52.518398,13.388788,NaN,10117,Berlin,Dussmann das KulturKaufhaus,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4adf61aef964a520177a21e3-1,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",4adf61aef964a520177a21e3,Behrenstr. 55,DE,Berlin,Deutschland,NaN,...,"[{'label': 'display', 'lat': 52.51656861215425...",52.516569,13.388008,NaN,10117,Berlin,Cookies Cream,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-562a9474498e20b9ac65c6fe-2,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",562a9474498e20b9ac65c6fe,Friedrichstr. 90,DE,Berlin,Deutschland,NaN,...,"[{'label': 'display', 'lat': 52.51822284368067...",52.518223,13.389239,NaN,10117,Berlin,Dussmann English Bookshop,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b4b65d9f964a5201a9a26e3-3,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",4b4b65d9f964a5201a9a26e3,Friedrichstr. 83,DE,Berlin,Deutschland,NaN,...,"[{'label': 'display', 'lat': 52.51569793249014...",52.515698,13.389298,NaN,10117,Berlin,COS,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4adcda8af964a520bd4921e3-4,"[{'id': '4bf58dd8d48988d136941735', 'name': 'O...",4adcda8af964a520bd4921e3,Behrenstr. 55-57,DE,Berlin,Deutschland,NaN,...,"[{'label': 'display', 'lat': 52.51596828902978...",52.515968,13.386701,Unter den Linden,10117,Berlin,Komische Oper,0,[],NaN


In [8]:
filtered_columns = ['venue.id', 'venue.location.lat', 'venue.location.lng', 'venue.name', 'venue.categories']
df_venues = berlin_venues.loc[:, filtered_columns]

In [9]:
df_venues.head(5)

,venue.id,venue.location.lat,venue.location.lng,venue.name,venue.categories
0,4adcda8ef964a520a74a21e3,52.518398,13.388788,Dussmann das KulturKaufhaus,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B..."
1,4adf61aef964a520177a21e3,52.516569,13.388008,Cookies Cream,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V..."
2,562a9474498e20b9ac65c6fe,52.518223,13.389239,Dussmann English Bookshop,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B..."
3,4b4b65d9f964a5201a9a26e3,52.515698,13.389298,COS,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C..."
4,4adcda8af964a520bd4921e3,52.515968,13.386701,Komische Oper,"[{'id': '4bf58dd8d48988d136941735', 'name': 'O..."


In [10]:
df_venues = pd.DataFrame(df_venues)

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
df_venues['venue.categories'] = df_venues.apply(get_category_type, axis=1)

# clean columns
df_venues.columns = [col.split(".")[-1] for col in df_venues.columns]


In [13]:
df_venues['categories'].value_counts()

Hotel                            11
History Museum                    4
Theater                           4
Coffee Shop                       4
German Restaurant                 4
Café                              3
Asian Restaurant                  3
Concert Hall                      3
Gourmet Shop                      3
Plaza                             3
Drugstore                         2
Cosmetics Shop                    2
Bookstore                         2
Cocktail Bar                      2
Restaurant                        2
Monument / Landmark               2
Bar                               2
Wine Bar                          2
Hotel Bar                         2
Historic Site                     2
Chocolate Shop                    2
Opera House                       2
Clothing Store                    2
Department Store                  1
Furniture / Home Store            1
Spa                               1
Exhibit                           1
Hookah Bar                  

Remove rows that are not included in the restaurant/caffe/bar etc category

In [14]:
irrelevant = ['Boutique','Supermarket', 'Outdoor Sculpture','Exhibit', 'Drugstore','Furniture / Home Store', 
              'Church','Museum', 'Art Museum','Souvenir Shop','Optical Shop', 'Memorial Site', 'Department Store', 
              'Clothing Store', 'Monument / Landmark', 'Historic Site', 'Concert Hall', 'Gym / Fitness Center', 
              'History Museum', 'Cosmetics Shop']

In [15]:
df_relevant = df_venues[~df_venues['categories'].isin(irrelevant)]

In [16]:
print('There are {} distinct categories of units included in the analyis.'.format(len(df_relevant['categories'].value_counts())))
print('The dataset contains {} distinct units in Berlin that will be included in the clustering.'.format(df_relevant.shape[0]))

There are 35 distinct categories of units included in the analyis.
The dataset contains 70 distinct units in Berlin that will be included in the clustering.


In [17]:
df_relevant.head(5)

,id,lat,lng,name,categories
0,4adcda8ef964a520a74a21e3,52.518398,13.388788,Dussmann das KulturKaufhaus,Bookstore
1,4adf61aef964a520177a21e3,52.516569,13.388008,Cookies Cream,Vegetarian / Vegan Restaurant
2,562a9474498e20b9ac65c6fe,52.518223,13.389239,Dussmann English Bookshop,Bookstore
4,4adcda8af964a520bd4921e3,52.515968,13.386701,Komische Oper,Opera House
5,4adcda79f964a520874621e3,52.518553,13.386270,Windhorst,Cocktail Bar


<br>

**Checking JSON extraction from Foursquare for one venue id**

In [23]:
venue_id908 = df_relevant.loc[2, 'id'] #Dussmann das KulturKaufhaus

In [24]:
url3 = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(venue_id908,
CLIENT_ID,
CLIENT_SECRET,
VERSION)

In [27]:
res1 = requests.get(url3).json()
res1

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5bf929314434b9534b28e3da'},
 'response': {}}

In [ ]:
print(res1['response']['venue'].keys())

In [ ]:
stats = res1['response']['venue']

In [ ]:
stats = json_normalize(stats)

In [ ]:
stats.columns

<br>

**Mapping variables: PhotosCount, TipsCount, Rating, RatingSignals to all venues IDs**

In [30]:
validcols = ['photos.count', 'tips.count', 'rating','ratingSignals']

In [31]:
df_relevant2=df_relevant2.reset_index(drop=True)

In [ ]:
url01 = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(df_relevant2.loc[0,'id'], CLIENT_ID, CLIENT_SECRET, VERSION)
res01 = requests.get(url01).json()['response']['venue']
det01 = json_normalize(res01)
venue01 =det01.loc[:, validcols]


In [ ]:
url21 = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(df_relevant2.loc[20,'id'], CLIENT_ID, CLIENT_SECRET, VERSION)
res21 = requests.get(url21).json()['response']['venue']
det21 = json_normalize(res21)
venue21 =det21.loc[:, validcols]

url22 = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(df_relevant2.loc[21,'id'], CLIENT_ID, CLIENT_SECRET, VERSION)
res22 = requests.get(url22).json()['response']['venue']
det22 = json_normalize(res22)
venue22 =det22.loc[:, validcols]
                   
url23 = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(df_relevant2.loc[22,'id'], CLIENT_ID, CLIENT_SECRET, VERSION)
res23 = requests.get(url23).json()['response']['venue']
det23 = json_normalize(res23)
venue23 =det23.loc[:, validcols]
                   


While running the loop to get data 4square, I would always get the 429 error, running out of quota. 
I had to manually add data for all the venues extracted initially and populate.

In [39]:
df_all = pd.read_excel("dataset.xlsx")

In [40]:
df_all.head(5)

,id,lat,lng,name,categories,photos.count,tips.count,rating,ratingSignals
0,4adcda8ef964a520a74a21e3,52.518398,13.388788,Dussmann das KulturKaufhaus,Bookstore,614,187,9.4,1384
1,4adf61aef964a520177a21e3,52.516569,13.388008,Cookies Cream,Vegetarian / Vegan Restaurant,179,105,9.3,449
2,562a9474498e20b9ac65c6fe,52.518223,13.389239,Dussmann English Bookshop,Bookstore,64,12,9.2,114
3,4adcda8af964a520bd4921e3,52.515968,13.386701,Komische Oper,Opera House,213,25,8.7,206
4,4adcda79f964a520874621e3,52.518553,13.386270,Windhorst,Cocktail Bar,56,49,9.0,165


Get dummies for categories columns

In [55]:
df_all_dum = pd.get_dummies(df_all, columns = ['categories'])

In [56]:
df_all_dum.head(5)

,id,lat,lng,name,photos.count,tips.count,rating,ratingSignals,categories_Art Gallery,categories_Asian Restaurant,...,categories_Roof Deck,categories_Salad Place,categories_Sandwich Place,categories_Spa,categories_Steakhouse,categories_Sushi Restaurant,categories_Theater,categories_Vegetarian / Vegan Restaurant,categories_Wine Bar,categories_Wine Shop
0,4adcda8ef964a520a74a21e3,52.518398,13.388788,Dussmann das KulturKaufhaus,614,187,9.4,1384,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4adf61aef964a520177a21e3,52.516569,13.388008,Cookies Cream,179,105,9.3,449,0,0,...,0,0,0,0,0,0,0,1,0,0
2,562a9474498e20b9ac65c6fe,52.518223,13.389239,Dussmann English Bookshop,64,12,9.2,114,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4adcda8af964a520bd4921e3,52.515968,13.386701,Komische Oper,213,25,8.7,206,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4adcda79f964a520874621e3,52.518553,13.386270,Windhorst,56,49,9.0,165,0,0,...,0,0,0,0,0,0,0,0,0,0


## Step 3: Clustering venues by category, no of photos, tips, ratings, and raters

In [57]:
berlin_clustering = df_all_dum.iloc[:,4:43]

In [58]:
berlin_clustering.head(5)

,photos.count,tips.count,rating,ratingSignals,categories_Art Gallery,categories_Asian Restaurant,categories_Austrian Restaurant,categories_Bar,categories_Bookstore,categories_Burrito Place,...,categories_Roof Deck,categories_Salad Place,categories_Sandwich Place,categories_Spa,categories_Steakhouse,categories_Sushi Restaurant,categories_Theater,categories_Vegetarian / Vegan Restaurant,categories_Wine Bar,categories_Wine Shop
0,614,187,9.4,1384,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,179,105,9.3,449,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,64,12,9.2,114,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,213,25,8.7,206,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,56,49,9.0,165,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
from sklearn import preprocessing

Scale data for kmeans

In [60]:
scaler = preprocessing.StandardScaler()
scaled_dfberlin = scaler.fit_transform(berlin_clustering)
scaled_dfberlin = pd.DataFrame(scaled_dfberlin)

In [61]:
scaled_dfberlin.head(5)

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,1.364408,1.068926,1.914636,3.558413,-0.120386,-0.211604,-0.120386,-0.171499,5.830952,-0.120386,...,-0.120386,-0.120386,-0.120386,-0.120386,-0.120386,-0.120386,-0.246183,-0.120386,-0.171499,-0.120386
1,-0.068171,0.347341,1.723446,0.569551,-0.120386,-0.211604,-0.120386,-0.171499,-0.171499,-0.120386,...,-0.120386,-0.120386,-0.120386,-0.120386,-0.120386,-0.120386,-0.246183,8.306624,-0.171499,-0.120386
2,-0.446899,-0.471042,1.532255,-0.501325,-0.120386,-0.211604,-0.120386,-0.171499,5.830952,-0.120386,...,-0.120386,-0.120386,-0.120386,-0.120386,-0.120386,-0.120386,-0.246183,-0.120386,-0.171499,-0.120386
3,0.043801,-0.356644,0.576303,-0.207234,-0.120386,-0.211604,-0.120386,-0.171499,-0.171499,-0.120386,...,-0.120386,-0.120386,-0.120386,-0.120386,-0.120386,-0.120386,-0.246183,-0.120386,-0.171499,-0.120386
4,-0.473245,-0.145448,1.149874,-0.338296,-0.120386,-0.211604,-0.120386,-0.171499,-0.171499,-0.120386,...,-0.120386,-0.120386,-0.120386,-0.120386,-0.120386,-0.120386,-0.246183,-0.120386,-0.171499,-0.120386


Run kmeans algorithm

In [62]:
kclusters = 4
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scaled_dfberlin)

In [63]:
kmeans.labels_[0:10] 

array([2, 2, 1, 1, 1, 2, 1, 1, 1, 2])

In [64]:
df_all['Cluster'] =  kmeans.labels_

In [65]:
df_all.head(5)

,id,lat,lng,name,categories,photos.count,tips.count,rating,ratingSignals,Cluster
0,4adcda8ef964a520a74a21e3,52.518398,13.388788,Dussmann das KulturKaufhaus,Bookstore,614,187,9.4,1384,2
1,4adf61aef964a520177a21e3,52.516569,13.388008,Cookies Cream,Vegetarian / Vegan Restaurant,179,105,9.3,449,2
2,562a9474498e20b9ac65c6fe,52.518223,13.389239,Dussmann English Bookshop,Bookstore,64,12,9.2,114,1
3,4adcda8af964a520bd4921e3,52.515968,13.386701,Komische Oper,Opera House,213,25,8.7,206,1
4,4adcda79f964a520874621e3,52.518553,13.386270,Windhorst,Cocktail Bar,56,49,9.0,165,1


## Step 4: Create a map of restaurants included in the analysis

In [67]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [69]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(df_all['lat'], df_all['lng'], df_all['name'], df_all['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters